# Install Libraries

In [ ]:
!pip install -q  einops accelerate  bitsandbytes langchain openai faiss-cpu tiktoken chromadb transformers

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: write).
Your token has been saved to /root/.c

In [ ]:
from langchain import HuggingFacePipeline
from transformers import AutoTokenizer
import transformers
import torch

# Prompt Template

In [ ]:
template = """
    Classify sentiments of the given question from the given list\
    list: ['anger', 'anticipation', 'disgust', 'fear', 'joy', 'sadness', 'surprise', 'trust', 'neutral'] \



    Question: And how is that different than all the times you risked me ? \
    Output: ['anticipation']

    Question: "And actresses are treated like I hate to use the word , but , shit ." \
    Output: ['anticipation', 'fear', 'joy', 'surprise']

    Question: {question}
    Output:
    """

#  Experiment 1 Sentiment Task

## ex 1.1 LLAMA 7b

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
model = "meta-llama/Llama-2-7b-chat-hf"

tokenizer = AutoTokenizer.from_pretrained(model)

pipeline = transformers.pipeline(
    "text-generation", #task
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.float16,
    trust_remote_code=True,
    device_map="auto",
    max_length=5000,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id
)

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [ ]:
llm = HuggingFacePipeline(pipeline = pipeline, model_kwargs = {'temperature':0})

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
import pandas as pd


In [ ]:
results=[]
def get_output(question):
  prompt = PromptTemplate.from_template(template)
  chain = LLMChain(llm=llm, prompt=prompt, verbose=True)
  result = chain({"question": question})
  print(result)
  results.append(result)
  return result



In [ ]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')
df = pd.read_csv("/content/drive/MyDrive/Untitled folder/filtered_sentiment_analysis_dataset.csv")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df.head()

,input,output_as_number,output
0,... And I don't think we need to discuss the T...,"[8, 1]","['neutral', 'anticipation']"
1,* So get up out of your bed,[1],['anticipation']
2,A confession that you hired [PERSON] ... and a...,"[1, 6]","['anticipation', 'surprise']"
3,A dead man has one half - hour to raise his ro...,[1],['anticipation']
4,A guy that's talking about he's gonna solve al...,[1],['anticipation']


In [ ]:
results = []
def get_output(question):
  prompt = PromptTemplate.from_template(template)
  chain = LLMChain(llm=llm, prompt=prompt, verbose=True)
  result = chain({"question": question})
  print(result)
  results.append(result)
  return result

In [ ]:
# Function to evaluate the string representation of lists into actual lists
def eval_list_string(s):
    try:
        return eval(s)
    except:
        return []

# Apply the function to the relevant columns
updated_df['text'] = updated_df['text'].apply(eval_list_string)
sentiment_df['output'] = sentiment_df['output'].apply(eval_list_string)

# Initialize the new columns
updated_df['exact_match'] = 0
updated_df['any_match'] = 0
updated_df['percent_match'] = 0.0

# We will only perform the operation for the first 2000 rows or the total rows if less than 2000
num_rows = min(2000, len(updated_df))

# Function to calculate the matches
def calculate_matches(row, sentiment_output_list):
    text_set = set(row['text'])
    sentiment_set = set(sentiment_output_list)

    # Exact match condition
    row['exact_match'] = int(text_set == sentiment_set)
    # Any match condition
    row['any_match'] = int(len(text_set.intersection(sentiment_set)) > 0)
    # Percent match condition
    if len(sentiment_set) > 0:
        row['percent_match'] = len(text_set.intersection(sentiment_set)) / len(sentiment_set)
    else:
        row['percent_match'] = 0.0
    return row

# Apply the match calculations for the first 2000 rows
for i in range(num_rows):
    updated_df.iloc[i] = calculate_matches(updated_df.iloc[i], sentiment_df['output'].iloc[i])

# Check the first few rows to verify the changes
updated_df_head_after_processing = updated_df.head()
updated_df_head_after_processing


In [ ]:
# Save the updated dataframe to a new CSV file
updated_file_path = 'content/updated_with_matches.csv'
updated_df.to_csv(updated_file_path, index=False)

updated_file_path


In [ ]:
import matplotlib.pyplot as plt

# Histogram for 'exact_match' values
plt.figure(figsize=(14, 5))

# Plot for exact match
plt.subplot(1, 3, 1)
updated_df['exact_match'].value_counts().plot(kind='bar', color=['skyblue', 'orange'])
plt.title('Histogram for Exact Match')
plt.xlabel('Exact Match')
plt.ylabel('Count')
plt.xticks(ticks=[0, 1], labels=['0', '1'], rotation='horizontal')

# Plot for any match
plt.subplot(1, 3, 2)
updated_df['any_match'].value_counts().plot(kind='bar', color=['skyblue', 'orange'])
plt.title('Histogram for Any Match')
plt.xlabel('Any Match')
plt.ylabel('Count')
plt.xticks(ticks=[0, 1], labels=['0', '1'], rotation='horizontal')

# Plot for percent match
plt.subplot(1, 3, 3)
updated_df['percent_match'].plot(kind='hist', bins=20, color='skyblue', edgecolor='black')
plt.title('Histogram for Percent Match')
plt.xlabel('Percent Match')
plt.ylabel('Count')

# Adjust layout to prevent overlap
plt.tight_layout()
plt.show()


In [ ]:
from collections import Counter

# Function to calculate the difference in emotions
def calculate_emotion_difference(row, sentiment_output_list):
    text_emotions = Counter(row['text'])
    sentiment_emotions = Counter(sentiment_output_list)
    # Calculate the difference
    difference = text_emotions - sentiment_emotions
    # Add the reverse difference to include emotions that are in sentiment but not in text
    difference += (sentiment_emotions - text_emotions)
    return difference

# Calculate emotion differences for the first 2000 rows
emotion_differences = []
for i in range(num_rows):
    difference = calculate_emotion_difference(updated_df.iloc[i], sentiment_df['output'].iloc[i])
    emotion_differences.append(difference)

# Combine all counters into one
total_difference_counter = sum(emotion_differences, Counter())

# Plot the difference
plt.figure(figsize=(10, 5))
plt.bar(total_difference_counter.keys(), total_difference_counter.values(), color='skyblue')
plt.title('Emotion Differences between Updated and Filtered Datasets')
plt.xlabel('Emotions')
plt.ylabel('Count Difference')
plt.xticks(rotation=45)
plt.show()

total_difference_counter


In [ ]:
# Calculate the number of predicted and actual emotions for each row
predicted_emotions_count = [len(row['text']) for _, row in updated_df.iterrows()][:50]
actual_emotions_count = [len(row) for row in sentiment_df['output']][:50]

# Define the labels and positions for the bar groups
labels = [f'Row {i+1}' for i in range(50)]
x = range(len(labels))  # the label locations

# Set up the matplotlib figure
plt.figure(figsize=(20, 10))

# Width of the bars
width = 0.35

# Plot bars
plt.bar(x, predicted_emotions_count, width, label='Predicted Emotions Count', color='skyblue')
plt.bar([p + width for p in x], actual_emotions_count, width, label='Actual Emotions Count', color='orange')

# Add some text for labels, title and custom x-axis tick labels, etc.
plt.ylabel('Count of Emotions')
plt.title('Count of Predicted vs Actual Emotions for Each Row')
plt.xticks([p + width / 2 for p in x], labels, rotation=90)
plt.legend()

# Show the plot
plt.tight_layout()
plt.show()


In [ ]:
# Load the newly uploaded datasets
llama_updated_df = pd.read_csv('content/LLama_updated_with_matches (1).csv')
original_sentiment_df = pd.read_csv('content/filtered_sentiment_analysis_dataset.csv')

# We will only perform the operation for the first 2000 rows or the total rows if less than 2000
num_rows_original = min(2000, len(original_sentiment_df))

# Evaluate string representations of lists into actual lists
llama_updated_df['text'] = llama_updated_df['text'].apply(eval_list_string)
original_sentiment_df['output'] = original_sentiment_df['output'].apply(eval_list_string)

# Calculate accuracy: where 'exact_match' is 1 for a row, the prediction is considered accurate
accuracy = (llama_updated_df['exact_match'].iloc[:num_rows_original].sum() / num_rows_original) * 100
accuracy


In [ ]:
# Generate a simple plot to visualize the accuracy
plt.figure(figsize=(6, 4))
accuracy_values = [accuracy, 100 - accuracy]
labels = ['Exact Matches', 'Non-Matches']
colors = ['skyblue', 'lightcoral']

# Plot
plt.bar(labels, accuracy_values, color=colors)
plt.title('Accuracy of Sentiment Predictions')
plt.ylabel('Percentage')
plt.ylim(0, 100)

# Add the text with the accuracy percentage on top of the bars
for i in range(len(accuracy_values)):
    plt.text(i, accuracy_values[i] + 1, f"{accuracy_values[i]:.2f}%", ha='center')

plt.tight_layout()
plt.show()
